In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE112154"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE112154"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE112154.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE112154.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE112154.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE112154_family.soft.gz', 'GSE112154_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE112154/GSE112154_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE112154/GSE112154_series_matrix.txt.gz
Background Information:
!Series_title	"Gene expression profiling of diffuse malignant peritoneal mesothelioma"
!Series_summary	"Diffuse malignant peritoneal mesothelioma (DMPM) is a rapidly lethal malignancy. The comprehension of the molecular features of DMPM is of utmost importance for the fruitful management of the disease, especially in patients who fail standard treatments and have a poor prognosis due to the lack of effective alternative therapeutic options."
!Series_overall_design	"Gene expression profiling was carried out on a series of 45 frozen surgical specimens of diffuse malignant peritoneal mesothelioma (DMPM), 3 normal peritoneum samples and 2 patient-derived cell lines."
Sample Characteristics Dictionary:
{0: ['sample type: norma

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any
import numpy as np

# This function will convert DMPM status value
def convert_trait(value: str) -> int:
    if value is None:
        return None
    
    # Extract actual value (after colon)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'DMPM frozen tumor specimen' in value:
        return 1  # Mesothelioma positive
    elif 'normal peritoneum' in value:
        return 0  # Mesothelioma negative
    else:
        return None  # DMPM cell lines are not considered in our analysis

# No age data is provided in the sample characteristics
convert_age = None

# No gender data is provided in the sample characteristics
convert_gender = None

# Determine data availability
trait_row = 0  # Data about DMPM status is available in row 0
age_row = None  # No age data is available
gender_row = None  # No gender data is available

# Check if gene expression data is available
# Based on the series title and overall design, this dataset contains gene expression profiles
is_gene_available = True

# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Only proceed with clinical feature extraction if trait data is available
if trait_row is not None:
    # Get the clinical data from the previous step
    clinical_data = pd.DataFrame({0: ['sample type: normal peritoneum', 'sample type: DMPM cell line', 'sample type: DMPM frozen tumor specimen']})
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Mesothelioma/clinical_data/GSE112154.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 47323
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina microarray probe IDs, not gene symbols
# These need to be mapped to human gene symbols for consistency with other datasets

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation dataframe
# From the preview, we can see that 'ID' contains the probe identifiers (ILMN_*) which match the gene expression data
# 'Symbol' column contains the gene symbols we want to map to

# 2. Get gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Show the first few rows of the mapping to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the resulting gene expression data
print("\nGene expression data preview after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")


Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}



Gene expression data preview after mapping:
Shape: (21464, 50)
First 5 gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (20259, 50)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE112154.csv
Number of samples: 50
Sample characteristics dictionary:
{0: ['sample type: normal peritoneum', 'sample type: DMPM cell line', 'sample type: DMPM frozen tumor specimen']}
Clinical data preview:
            Mesothelioma
GSM3058890             1
GSM3058891             1
GSM3058892             1
GSM3058893             1
GSM3058894             1
Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE112154.csv
Shape of linked data: (50, 20260)


Shape of linked data after handling missing values: (50, 20260)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Dataset validation failed due to biased trait variable. Final linked data not saved.
